In [4]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np 
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope # to include UNET_Functions when loading weight file
from tensorflow.keras.utils import get_custom_objects
from sklearn.metrics import f1_score
from UNET_Functions_IoU_DiceCoef_DiceLoss import DiceLoss, DiceCoef, IoU
from UNET_Lung_Segmentation_Train import CreateDir, LoadData, TfDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
import pandas as pd

import ipywidgets as widgets
from IPython.display import display

In [15]:
"""Global Parameters/Variables"""
H = 512
W = 512

In [16]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path) # creates path of none exist

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    xOrg = x
    x = x/255.0 # normalises
    x = x.astype(np.float32) # converts images into 32-bit floats
    x = np.expand_dims(x, axis=0) # (1, 256, 256, 3)
    return xOrg, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    xOrg = x
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return xOrg, x

def save_result(xOrg, yOrg, yPred, save_path):
    line = np.ones((H, 10, 3)) * 255 # 10px line to separate img and mask and predicted mask (segmentation result)

    yOrg = np.expand_dims(yOrg, axis=-1) # increases channels from 1 to 3
    yOrg = np.concatenate([yOrg, yOrg, yOrg], axis=-1) # (256, 256, 3)

    yPred = np.expand_dims(yPred, axis=-1)
    yPred = np.concatenate([yPred, yPred, yPred], axis=-1) * 255.0

    concatImages = np.concatenate([xOrg, line, yOrg, line, yPred], axis=1) # concatenates all imgs
    cv2.imwrite(save_path, concatImages)

if __name__ == "__main__":
    create_dir("FlippedEval_BaseTest")

    """ Load Model """
    with CustomObjectScope({'IoU': IoU, 'DiceCoef': DiceCoef, 'DiceLoss': DiceLoss}): # registers custom metrics
        model = tf.keras.models.load_model("TrainingFilesForFlipped/model.h5")

    """ Dataset """
    dataset_path = r"C:\Users\REDACTED NAME\Documents\PortModel\DatasetsForSciExt\DatasetNoiseUnet"
    (xTrain, yTrain), (xValid, yValid), (xTest, yTest) = LoadData(dataset_path)

    """ Prediction and metrics values """
    SCORE = []
    for x, y in tqdm(zip(xTest, yTest), total=len(xTest)): # creates progress bar
        name = x.split("\\")[-1]

        """ Reading the image and mask """
        xOrg, x = read_image(x)
        yOrg, y = read_mask(y)

        """ Prediction """
        yPred = model.predict(x)[0] > 0.5 # if val > 0.5 then = 1, > 0.5 then = 0
        yPred = np.squeeze(yPred, axis=-1) # converts binary mask into int vals
        yPred = yPred.astype(np.int32) # converts binary mask into int vals

        savePath = f"FlippedEval_BaseTest/{name}" # path in which img is saved
        save_result(xOrg, yOrg, yPred, savePath)

        """ Flattening the numpy arrays. """
        y = y.flatten()
        yPred = yPred.flatten()

        """ Calculating metrics values """
        accValue = accuracy_score(y, yPred)
        f1Value = f1_score(y, yPred, labels=[0, 1], average="binary")
        recallValue = recall_score(y, yPred, labels=[0, 1], average="binary")
        precisionValue = precision_score(y, yPred, labels=[0, 1], average="binary")
        SCORE.append([name, accValue, f1Value, recallValue, precisionValue])

    """ Metrics values """
    score = [s[1:]for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Recall: {score[2]:0.5f}")
    print(f"Precision: {score[3]:0.5f}")

    """ Saving all the results """
    df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Recall", "Precision"])
    df.to_csv("FlippedEval_BaseTest/FlippedMetrics_BaseTest.csv")

60


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.05it/s]

Accuracy: 0.71909
F1: 0.08668
Recall: 0.05815
Precision: 0.65789
